In [ ]:
import geopandas as gpd
import healpy as hp
import numpy as np
import pandas as pd
import shapely

import healpix_convolution.neighbours as nb

In [ ]:
def boundary_vectors(nside, nest, cell_ids):
    flattened = np.reshape(cell_ids, -1)

    vertex_vectors = hp.boundaries(nside, flattened, nest=nest)

    reshaped = np.reshape(vertex_vectors, cell_ids.shape + vertex_vectors.shape[1:])

    return np.moveaxis(reshaped, -2, -1)


def vectors_to_geographic(vectors):
    reshaped = np.reshape(vectors, (-1, 3))

    lon, lat = hp.vec2ang(vectors, lonlat=True)

    lon_ = np.reshape(lon, vectors.shape[:-1])
    lat_ = np.reshape(lat, vectors.shape[:-1])

    return lon_, lat_

In [ ]:
def visualize_cells(cell_ids, resolution, indexing_scheme):
    flattened = np.reshape(cell_ids, -1)

    boundary_vecs = boundary_vectors(
        2**resolution, indexing_scheme == "nested", flattened
    )
    boundaries = np.stack(vectors_to_geographic(boundary_vecs), axis=-1)
    polygons = shapely.polygons(boundaries)

    df = gpd.GeoDataFrame(index=flattened, geometry=polygons, crs=4326)
    return df

In [ ]:
import folium

In [ ]:
def visualize_neighbours(neighbours, resolution, indexing_scheme):
    nside = 2**resolution
    nest = indexing_scheme == "nested"
    vectors = boundary_vectors(nside, nest, neighbours)
    boundaries = np.stack(vectors_to_geographic(vectors), axis=-1)
    polygons = shapely.polygons(boundaries)

    centers = polygons[:, 0]
    n_ = polygons[:, 1:]

    m = folium.Map()

    for polygon, cell_id in zip(np.ravel(centers), np.ravel(neighbours[:, 0])):
        if cell_id == -1:
            continue
        folium.GeoJson(
            shapely.to_geojson(polygon),
            tooltip=f"cell id: {cell_id}",
        ).add_to(m)

    for polygon, cell_id in zip(np.ravel(n_), np.ravel(neighbours[:, 1:])):
        if cell_id == -1:
            continue
        folium.GeoJson(
            shapely.to_geojson(polygon),
            color="red",
            tooltip=f"cell id: {cell_id}",
        ).add_to(m)

    return m

In [ ]:
resolution = 1
cell_ids = np.array([25], dtype="int64")
neighbours = nb.neighbours(
    cell_ids, resolution=resolution, indexing_scheme="nested", ring=2
)

neighbours

In [ ]:
visualize_neighbours(neighbours, resolution, indexing_scheme="nested")

In [ ]:
resolution = 2
cell_ids = np.arange(12 * 4**resolution, dtype="int64")

df = visualize_cells(cell_ids, resolution, indexing_scheme="nested")
df.explore()

In [ ]:
def visualize_cells(center_cells, neighbours, resolution, indexing_scheme):
    flattened = np.reshape(neighbours, -1)
    boundary_vecs = boundary_vectors(
        2**resolution, indexing_scheme == "nested", flattened
    )
    boundaries = np.stack(vectors_to_geographic(boundary_vecs), axis=-1)
    polygons = shapely.polygons(boundaries)

    neighbor_id = np.arange(9)
    index = pd.MultiIndex.from_product((center_cells, neighbor_id))

    df = gpd.GeoDataFrame(index=index, geometry=polygons, crs=4326)
    return df


resolution = 2
cell_ids = np.arange(4**resolution, 2 * 4**resolution)

actual = nb.neighbours(cell_ids, resolution=resolution, indexing_scheme="nested")
expected_ = hp.get_all_neighbours(2**resolution, cell_ids, nest=True).T
expected = np.concatenate([cell_ids[:, None], expected_], axis=1)

In [ ]:
index = 2
df1 = visualize_cells(cell_ids, actual, resolution, indexing_scheme="nested")
df1.loc[4**resolution + index].explore()

In [ ]:
df2 = visualize_cells(cell_ids, expected, resolution, indexing_scheme="nested")
df2.loc[4**resolution + index].explore()

In [ ]:
resolution = 12
cell_ids = np.arange(12 * 4**resolution)
ring = 4

In [ ]:
cell_ids.nbytes / 1e9

In [ ]:
nb.neighbours(cell_ids, resolution=resolution, indexing_scheme="nested", ring=3)